# 3 함수, 반복자, 제네레이터
함수형 프로그래밍의 핵심은 순수 함수를 사용해 입력 정의역(domain)의 값을 출력 치역(range) 값으로 바꾸는 것이다. 

순수 함수에는 부수 효과(외부의 상태를 변경 또는 함수로 들어온 인자의 상태를 직접 변경하는 것)가 없으며 따라서 변수 의존 관계를 줄일 수 있다. 파이썬에서 대입문을 완전히 없앨 수는 없겠지만, 상태가 있는 객체에 의존하지 않을 수 있다. 이를 위해서 파이썬이 제공하는 내장 데이터 구조 중 상태가 있는 연산을 필요로 하지 않는 구조를 활용해야 한다.

따라서 이번 장은 파이썬의 기능 몇 가지를 살펴볼 것이다.

* 부수 효과가 없는 순수 함수

* 인자로 넘길 수 있거나 함수의 결과로 반환할 수 있는 객체인 함수

* 객체지향적인 후위 표기법이나 전위 표기법으로 파이썬 문자열 사용

* tuple이나 namedtuple을 사용해 상태가 없는 객체 생성

* 반복 가능한 컬렉션을 함수형 프로그래밍을 위한 주 설계 도구로 활용

2장에서 살폈듯 파이썬은 재귀 깊이에 제한을 두며, 자동으로 TCO를 진행하지 않는다. 따라서 제너레이터 식을 이용해 직접 재귀를 최적화해야 한다. 여기서는 다음과 같은 작업을 수행하는 제너레이터 식을 작성할 것이다.

* 변환

* 재구성 

* 복잡한 계산


---


* 객체의 상태: 상태는 **특정 시점에 객체가 가지고 있는 정보의 집합**으로 객체의 구조적 특징을 표현한다. 객체의 상태는 객체에 존재하는 정적인 프로퍼티와 동적인 프로퍼티 값으로 구성된다. 객체의 프로퍼티는 단순한 값과 다른 객체를 참조하는 링크로 구분할 수 있다.


---

## 순수 함수 작성하기
순수 함수는 부수 효과가 없다. 즉, 변수의 전역적인 상태를 변경하는 일이 결코 없다. global문을 사용하지 않는다면 이러한 목표를 거의 충족할 수 있을 것이다. 또한 상태를 바꿀 수 있는 객체를 다루는 방식을 바꿔야만 한다. 자유 변수(free variable)를 사용해 파이썬 전역에 있는 값을 참조하면 매개변수를 적절히 사용해 처리할 수 있다.

다음은 전역적인 문장의 사용 예를 보여주는 예제다.

In [1]:
def some_function(a, b, t):
    
    return a+b+t+global_adjustment

위 예제에서 함수를 리펙토링하여 global_adjustment 변수를 적절한 매개변수로 바꿀 수 있다. 하지만 이런 작업은 복잡한 애플리케이션에서 많은 파장을 불러일으킬 수 있다.

상태가 있는 파이썬 내부 객체는 많다. file 클래스의 인스턴스나 모든 file-like 객체가 자주 사용되는 '상태가 있는 객체'다. 상태가 있는 파일 객체를 잘 정의된 영역 안으로 제한할 수 있는 **with문**을 잘 활용하는 것이 중요하다.(파일 객체는 항상 with 컨텍스트 안에서 사용하라)

전역 파일 객체, 전역 데이터베이스 연결 등을 피하고 그와 관련 있는 상태를 피해야 한다. 전역 파일 객체는 열린 파일을 차리하는 경우에 매우 흔하게 사용하는 패턴이다. 다음 코드를 보자.

In [ ]:
def open_(iname, oname):
    
    global ifile, ofile
    
    ifile = open(iname, "r")
    ofile = open(onamem "w")

위 코드는 다양한 다른 함수들이 ifile이나 ofile 변수를 사용할 수 있도록 열어두었다. 이는 좋은 설계가 아니며 피해야 한다. 파일은 함수에 제공되는 매개변수여야 하며, 열린 파일은 with문으로 감싸서 상태에 따른 동작을 제대로 처리하도록 설계해야 한다.


---

## first class 객체인 함수
함수가 속성(attribute)가 있는 객체이기 때문에 우리는 __name__이나 __name__ 속성을 사용해 docstring이나 함수의 이름을 뽑아낼 수 있다. 또한 함수의 본문을 __code__ 속성을 이용해 가져올 수도 있다. 

함수를 변수에 대입하거나, 함수를 인자로 넘기거나, 함수를 값으로 반환할 수도 있다. 이러한 기법을 이용해 고차 함수를 쉽게 만들 수도 있다.

이와 아울러 호출 가능한 객체를 사용해 함수를 정의할 수도 있다. 호출 가능 객체 역시 first class 객체다. 심지어 호출 가능 클래스 정의를 고차 함수로 볼 수도 있다. 다만 호출 가능 객체에서 __init()__ 메서드를 사용하는 방식은 신중하게 생각해야 한다. 한 가지 일반적인 응용 방법은 __init()__ 메서드를 사용해 전략 디자인 패턴(Strategy design pattern)에 부합하는 객체를 만드는 것이다. 

전략 디자인 패턴을 따르는 클래스는 알고리즘이나 알고리즘 일부를 제공하는 다른 객체에 의존한다. 이 패턴은 알고리즘의 자세한 부분을 클래스 안에 컴파일해 넣는 대신, 실행 시점에 알고리즘의 세부 사항을 주입할 수 있게 해준다.

아래는 내장된 전략 객체가 있는 호출 가능한 객체 예제다.

In [2]:
import collections

class Mersennel(collections.Callable):
    
    def __init__(self, algorithm):
        
        # 주어진 인자만큼 2를 거듭제곱하는 함수를 참조
        self.pow2 = algorithm
        
    def __call__(self, arg):
        
        # 인자를 받아서 함수를 적용한 뒤 나온 값에 1을 뺀 값을 반환
        return self.pow2(arg) - 1

/var/folders/p3/fjkwnjwd4dzcfcqt47dlgrm40000gn/T/ipykernel_14196/7786008.py:3: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  class Mersennel(collections.Callable):


이 클래스는 __init__()를 사용해 다른 함수 참조를 저장한다. 하지만 아무런 상태가 있는 인스턴스 변수를 만들지 않는다. 이 클래스에 끼워 넣을 수 있는 세 가지 후보 객체는 다음과 같다.

In [4]:
# 주어진 인자만큼 2를 거듭제곱하는 함수(b: 지수)


# 비트 left shift 연산으로 2의 거듭제곱을 계산한다.
def shifty(b):
    
    # 비트 1을 왼쪽으로 shift할 떄마다 2의 거듭제곱만큼 값이 커지는 것
    return 1 << b


# 재귀를 이용해 2의 거듭제곱을 계산한다. 인자로 받은 값에서 -1씩 줄어들며 0이 되면 끝난다.
def multy(b):
    
    # 기저 조건
    if b == 0:
        
        return 1
    
    # 2*multy(b-1)=2*2*multy(b-2)=...=2^b
    return 2*multy(b-1)


# 재귀함수
# faster 함수는 분할 정복(divide and conquer) 전략을 사용해 b번이 아니라 log_2(b)번의 곱셈을 수행한다.(시간 효율성)
# 과정에서 b를 계속 2씩 나누는 대신, t*t로 지수는 *2가 되어 보상된다.
def faster(b):
    
    # 기저 조건
    if b == 0:
        
        return 1
    
    # b가 홀수면
    if b % 2 == 1:
        
        # 홀수 재귀
        # 홀수 턴에서는 2^1만큼 늘어난다.
        return 2*faster(b-1)
    
    # b가 짝수면
    # 짝수 재귀(log_2(b)번의 곱셈을 수행하게 만드는 핵심)
    # t: b를 2로 나눈 몫을 faster() 함수 인자로 넣어서 실행한다.
    # 짝수는 2로 나눠주는 만큼, 반환값도 ()^2로 지수가 2배로 늘어난다.
    t = faster(b//2)
    
    return t*t

알고리즘 전략을 내장한 Mersennel 클래스의 인스턴스는 다음과 같이 만들 수 있다.

In [5]:
# 첫 번째 객체
mls= Mersennel(shifty)

# 두 번째 객체
mlm= Mersennel(multy)

# 세 번째 객체
mlf= Mersennel(faster)

---